<a href="https://colab.research.google.com/github/yeonghun00/stock_public/blob/main/issue_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import datetime
import html
import re

In [2]:
# market 
kospi_url = 'http://kind.krx.co.kr/corpgeneral/corpList.do?method=download&searchType=13&marketType=%sstockMkt'
kosdaq_url = 'http://kind.krx.co.kr/corpgeneral/corpList.do?method=download&searchType=13&marketType=%skosdaqMkt'
kospi_df = pd.read_html(kospi_url, header=0)[0]
kosdaq_df = pd.read_html(kosdaq_url, header=0)[0]
df = pd.concat([kospi_df, kosdaq_df], ignore_index=True)
df.head()

,회사명,종목코드,업종,주요제품,상장일,결산월,대표자명,홈페이지,지역
0,DRB동일,4840,고무제품 제조업,"고무벨트(V벨트,콘베이어벨트,평벨트),프라스틱제품 제조,판매",1976-05-21,12월,류영식,http://drbworld.com,부산광역시
1,DSR,155660,1차 비철금속 제조업,합섬섬유로프,2013-05-15,12월,홍석빈,http://www.dsr.com,부산광역시
2,GS글로벌,1250,상품 종합 도매업,"수출입업(시멘트,철강금속,전기전자,섬유,기계화학),상품중개,광업,채석업/하수처리 서...",1976-06-26,12월,김태형,http://www.gsgcorp.com,서울특별시
3,HDC현대산업개발,294870,건물 건설업,"외주주택, 자체공사, 일반건축, 토목 등",2018-06-12,12월,"권순호, 정경구",http://www.hdc-dvp.com,서울특별시
4,KEC,92220,반도체 제조업,반도체 제조,2006-10-16,12월,박남규 김학남,http://www.kec.co.kr,서울특별시


In [3]:
# 특징주
def get_search(keyword='%C6%AF%C2%A1%C1%D6', start_date='1', end_date='1'):
  start_date = str(datetime.datetime.now().date().strftime("%Y-%m-%d")) if start_date == '1' else start_date
  end_date = str(datetime.datetime.now().date().strftime("%Y-%m-%d")) if end_date == '1' else end_date

  li = []
  for page in range(1,50000):
    url = 'https://finance.naver.com/news/news_search.nhn?rcdate=&q=' + keyword +\
    '&sm=title.basic&pd=4&stDateStart=' + start_date + '&stDateEnd=' + end_date + '&page=' + str(page)
    result = requests.get(url, headers = {'User-Agent' : 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.121 Safari/537.36'})
    bs_obj = BeautifulSoup(result.content, "html.parser")
    text = [x.get_text() for x in bs_obj.find_all(re.compile('d*'), {'class':"articleSubject"})] 
    if not text:
      break
    li += text

  return li

In [4]:
text_li = get_search('%C6%AF%C2%A1%C1%D6','2021-06-07', '2021-06-11')

In [5]:
def get_word_lis(li):
  text_lis = [x.split() for x in li]
  word_lis = []

  for text_li in text_lis:
    for i in ['[',']', '·', '(', ')', '‘', '’','\'', '"', ',','...','…', '특징주']:
      text_li = [x.replace(i, ' ') for x in text_li]
    word_lis.append((' '.join(text_li)).split())

  return word_lis

In [6]:
from collections import Counter

def get_freq_codes(li):
  flatten_news_words = [j for sub in li for j in sub]
  news_words_frq = Counter(flatten_news_words).most_common()

  set_codes = set(df['회사명'].values)

  freq_codes = []
  for i in range(len([x[0] for x in news_words_frq])):
    if news_words_frq[i][0] in set_codes:
      freq_codes.append(news_words_frq[i])

  return freq_codes

In [7]:
word_lis = get_word_lis(text_li)
freq_codes = get_freq_codes(word_lis)
freq_codes

[('두산중공업', 15),
 ('하림', 9),
 ('쌍방울', 9),
 ('엘비루셈', 8),
 ('만도', 7),
 ('에이치엘비파워', 7),
 ('신풍제약', 6),
 ('대한전선', 6),
 ('SK', 4),
 ('포스코', 4),
 ('헬릭스미스', 4),
 ('두산퓨얼셀', 4),
 ('SK텔레콤', 4),
 ('카카오', 4),
 ('SBS', 4),
 ('효성', 3),
 ('넥스턴바이오', 3),
 ('삼성SDI', 3),
 ('호텔신라', 3),
 ('쎄미시스코', 3),
 ('HMM', 3),
 ('코스맥스엔비티', 3),
 ('삼성바이오로직스', 3),
 ('비트컴퓨터', 2),
 ('현대이지웰', 2),
 ('현대바이오', 2),
 ('LG화학', 2),
 ('유니온머티리얼', 2),
 ('웰바이오텍', 2),
 ('차바이오텍', 2),
 ('KBI메탈', 2),
 ('퓨쳐스트림네트웍스', 2),
 ('대원전선', 2),
 ('참좋은여행', 2),
 ('대한항공', 2),
 ('포시에스', 2),
 ('SK하이닉스', 2),
 ('파워로직스', 2),
 ('서전기전', 2),
 ('영화테크', 2),
 ('한양증권', 2),
 ('셀리드', 2),
 ('한국전자인증', 2),
 ('알테오젠', 2),
 ('플레이위드', 2),
 ('한국가스공사', 2),
 ('기아', 2),
 ('파멥신', 2),
 ('로지시스', 1),
 ('네오티스', 1),
 ('가온미디어', 1),
 ('평화산업', 1),
 ('넥스트사이언스', 1),
 ('삼보산업', 1),
 ('LG헬로비전', 1),
 ('포스코엠텍', 1),
 ('바이오톡스텍', 1),
 ('엘티씨', 1),
 ('나노스', 1),
 ('뉴인텍', 1),
 ('동아화성', 1),
 ('에스에너지', 1),
 ('데브시스터즈', 1),
 ('바이넥스', 1),
 ('서플러스글로벌', 1),
 ('룽투코리아', 1),
 ('한온시스템', 1),
 ('현대백화점', 1),
 ('KNN',